In [118]:
import torch
import math
import torch.nn as nn
from torch.nn import functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from decode_patterns.create_images import create_images, crop_data, train_test
import numpy as np

In [119]:
batch_size = 32
num_epochs = 500
num_epochs = int(num_epochs)

In [110]:
# prepare data
limit = 9600

drum, bass, tempo = create_images(file_name="../patterns_pairs.tsv", limit=limit)

In [189]:
# model class with bottle throat
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 128 * 14 (+16 with cond)= 1792 --> 2048
        self.fc11 = nn.Linear(input_dim, 1500)
        #nn.init.normal_(self.fc11.weight, mean=1, std=1)
#     self.fc1.weight = nn.Parameter(self.fc1.weight * math.sqrt(input_dim/2))
        self.s11 = nn.Sigmoid()
        
        self.fc12 = nn.Linear(1500, 1100)
        #nn.init.normal_(self.fc12.weight, mean=1, std=1)
        self.s12 = nn.Sigmoid()
        
        self.fc13 = nn.Linear(1100, 700)
        #nn.init.normal_(self.fc13.weight, mean=1, std=1)
        self.s13 = nn.Sigmoid()
        
        self.fc24 = nn.Linear(700, 1400)
        #nn.init.normal_(self.fc24.weight, mean=1, std=1) 
        self.s24 = nn.Sigmoid()
        
        self.fc25 = nn.Linear(1400, 2000)
        #nn.init.normal_(self.fc25.weight, mean=1, std=1) 
        self.s25 = nn.Sigmoid()
        
        self.fc26 = nn.Linear(2000, 4608)
        #nn.init.normal_(self.fc26.weight, mean=1, std=1) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        out = self.fc11(x)
        out = self.s11(out)
        
        out = self.fc12(out)
        out = self.s12(out)
        
        out = self.fc13(out)
        out = self.s13(out)
        
        out = self.fc24(out)
        out = self.s24(out)
        
        out = self.fc25(out)
        out = self.s25(out)
        
        out = self.fc26(out)
        out = self.sigmoid(out)
        
        return out

In [153]:
# model class with bottle throat
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 128 * 14 (+16 with cond)= 1792 --> 2048
        self.fc11 = nn.Linear(input_dim, 1500)
        nn.init.normal_(self.fc11.weight, mean=1, std=1)
#     self.fc1.weight = nn.Parameter(self.fc1.weight * math.sqrt(input_dim/2))
        self.relu11 = nn.ReLU()
        
        self.fc12 = nn.Linear(1500, 1100)
#         nn.init.normal_(self.fc12.weight, mean=1, std=1)
        self.relu12 = nn.ReLU()
        
        self.fc13 = nn.Linear(1100, 700)
#         nn.init.normal_(self.fc13.weight, mean=1, std=1)
        self.relu13 = nn.ReLU()
        
        self.fc14 = nn.Linear(700, 400)
#         nn.init.normal_(self.fc14.weight, mean=1, std=1) 
        self.relu14 = nn.ReLU()
        
        self.fc15 = nn.Linear(400, 130)
#         nn.init.normal_(self.fc15.weight, mean=1, std=1) 
        self.relu15 = nn.ReLU()
        
        
        # throat
        self.fc16 = nn.Linear(130, 6)
#         nn.init.normal_(self.fc16.weight, mean=1, std=1) 
        self.relu16 = nn.ReLU()
        
        self.fc21 = nn.Linear(6, 130)
#         nn.init.normal_(self.fc21.weight, mean=1, std=1) 
        self.relu21 = nn.ReLU()
        
        
        
        self.fc22 = nn.Linear(130, 400)
#         nn.init.normal_(self.fc22.weight, mean=1, std=1) 
        self.relu22 = nn.ReLU()
        
        self.fc23 = nn.Linear(400, 700)
#         nn.init.normal_(self.fc23.weight, mean=1, std=1) 
        self.relu23 = nn.ReLU()
        
        self.fc24 = nn.Linear(700, 1400)
#         nn.init.normal_(self.fc24.weight, mean=1, std=1) 
        self.relu24 = nn.ReLU()
        
        self.fc25 = nn.Linear(1400, 2000)
#         nn.init.normal_(self.fc25.weight, mean=1, std=1) 
        self.relu25 = nn.ReLU()
        
        self.fc26 = nn.Linear(2000, 4608)
#         nn.init.normal_(self.fc26.weight, mean=1, std=1) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        out = self.fc11(x)
        out = self.relu11(out)
        
        out = self.fc12(out)
        out = self.relu12(out)
        
        out = self.fc13(out)
        out = self.relu13(out)
        
        out = self.fc14(out)
        out = self.relu14(out)
        
        out = self.fc15(out)
        out = self.relu15(out)
        
        out = self.fc16(out)
        out = self.relu16(out)
        
    
        
        out = self.fc21(out)
        out = self.relu21(out)
        
        out = self.fc22(out)
        out = self.relu22(out)
        
        out = self.fc23(out)
        out = self.relu23(out)
        
        out = self.fc24(out)
        out = self.relu24(out)
        
        out = self.fc25(out)
        out = self.relu25(out)
        
        out = self.fc26(out)
        out = self.sigmoid(out)
        
        return out


In [16]:
# model class
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 128 * 14 (+16 with cond)= 1792 --> 2048
        # веса накидываются тут
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # решение по весам
#         nn.init.normal_(self.fc1.weight, mean=0, std=1)
#         self.fc1.weight = nn.Parameter(self.fc1.weight * math.sqrt(input_dim/2))
        self.relu1 = nn.ReLU()
        
        # Linear function 2: 2048 --> 2048
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         # Non-linearity 2
#         nn.init.normal_(self.fc2.weight, mean=0, std=1)
#         self.fc2.weight = nn.Parameter(self.fc2.weight * math.sqrt(hidden_dim/2))
        self.relu2 = nn.ReLU()

        # Linear function 3: 2048 --> 2048
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        nn.init.normal_(self.fc3.weight, mean=0, std=1)
        self.fc3.weight = nn.Parameter(self.fc3.weight * math.sqrt(hidden_dim/2))
        self.relu3 = nn.ReLU()

        # Linear function 4 (readout): 2048 --> 128 * 36 = 4608
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
        nn.init.normal_(self.fc4.weight, mean=0, std=1)
        self.fc4.weight = nn.Parameter(self.fc4.weight * math.sqrt(hidden_dim/2))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)

        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.relu3(out)

        # Linear function 4 (readout)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out


In [190]:
# instantiate model class

input_dim = 128 * 14 + 16
output_dim = 128 * 36
hidden_dim = 2048
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)


#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model.to(device)

FeedforwardNeuralNetModel(
  (fc11): Linear(in_features=1808, out_features=1500, bias=True)
  (s11): Sigmoid()
  (fc12): Linear(in_features=1500, out_features=1100, bias=True)
  (s12): Sigmoid()
  (fc13): Linear(in_features=1100, out_features=700, bias=True)
  (s13): Sigmoid()
  (fc24): Linear(in_features=700, out_features=1400, bias=True)
  (s24): Sigmoid()
  (fc25): Linear(in_features=1400, out_features=2000, bias=True)
  (s25): Sigmoid()
  (fc26): Linear(in_features=2000, out_features=4608, bias=True)
  (sigmoid): Sigmoid()
)

In [191]:
# instantiate loss class
# criterion = nn.MSELoss()
# criterion = nn.BCEWithLogitsLoss()
criterion = F.binary_cross_entropy

In [192]:
learning_rate = 0.001

In [193]:
# instantiate optimizer class
# менять learning_rate по мере обучения

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# initialise weights

In [194]:
k = 0
l = 0
sz = 0
iter = 0
iter_epoch = 0
for epoch in range(num_epochs):
    (train_set, train_t), (test_set, test_t) = train_test(drum, bass, tempo, batch_size=batch_size, img_size=(128, 50))
    iter_epoch += 1
    print(f"Epoch # {iter_epoch}")
    for i, (images, labels) in enumerate(zip(*train_set)):
        
        images = images.view(-1, input_dim).requires_grad_().to(device).float()
        labels = labels.to(device).float().reshape((32, 4608))

        optimizer.zero_grad()

        outputs = model(images)

#       print("images.size(): ", images.size(), "\nlabels.size(): ", labels.size(), "\noutputs.size(): ", 
#               outputs.size(), "\noutputs[i].size()", outputs[i].size(), "\nlabels.view(-1, output_dim)[i].size(): ",
#              labels.view(-1, output_dim)[i].size())
#       есть тип ошибок, которые не нужно считать в цикле
#       loss = criterion(outputs, labels.view(-1, output_dim).long())
        loss = 0
        for i in range(batch_size):
            loss += criterion(outputs[i], labels[i])
#         loss = criterion(outputs.view(-1, output_dim), labels.squeeze().view(-1, output_dim))
        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 100 == 0:
            error = 0
            for img, lbl in zip(*test_set):
            # Calculate Accuracy         
                # Iterate through test dataset
                out = model(img.view(-1, input_dim).to(device).float())
                error += criterion(out.float(), lbl.float())
            print('Iteration: {}. error: {}'.format(iter, error))

Epoch # 1
Iteration: 100. error: 47.64019012451172
Iteration: 200. error: 47.44186782836914
Epoch # 2
Iteration: 300. error: 47.11676788330078
Iteration: 400. error: 46.95383071899414
Epoch # 3
Iteration: 500. error: 46.69102478027344
Iteration: 600. error: 46.734989166259766
Iteration: 700. error: 46.661067962646484
Epoch # 4
Iteration: 800. error: 48.328330993652344
Iteration: 900. error: 48.21558380126953
Epoch # 5
Iteration: 1000. error: 46.243228912353516
Iteration: 1100. error: 46.187259674072266
Iteration: 1200. error: 46.13276290893555
Epoch # 6
Iteration: 1300. error: 46.46315383911133
Iteration: 1400. error: 46.41460037231445
Epoch # 7
Iteration: 1500. error: 47.680084228515625
Iteration: 1600. error: 47.67027282714844
Epoch # 8
Iteration: 1700. error: 46.316776275634766
Iteration: 1800. error: 46.390892028808594
Iteration: 1900. error: 46.31707763671875
Epoch # 9
Iteration: 2000. error: 46.7801399230957
Iteration: 2100. error: 46.785194396972656
Epoch # 10
Iteration: 2200. e

KeyboardInterrupt: 

In [160]:
print(k, l, sz, batch_size)


torch.Size([32, 4608]) torch.Size([4608]) torch.Size([32, 4608]) 32


In [201]:
outputs = model(test_set[0].view(-1, 128 * 14 + 16).to(device).float())
lables = test_set[1].view(-1, 128 * 36).to(device)
print(lables.max())
print(lables.min())
print(lables.mean())

tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0.0050, dtype=torch.float64)


In [202]:
(train_set, _), (test_set, _) = train_test(drum, bass, tempo, batch_size = batch_size)

In [14]:
torch.save(model.state_dict(), "../model_2_state")

In [6]:
model = FeedforwardNeuralNetModel(input_dim - 16, hidden_dim, output_dim)
model.to(device)
model.load_state_dict(torch.load("../model_0_state"))
model.eval()

FeedforwardNeuralNetModel(
  (fc1): Linear(in_features=1792, out_features=2048, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=2048, out_features=2048, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=2048, out_features=4608, bias=True)
  (sigmoid): Sigmoid()
)

In [203]:
(train_set, train_tempo), (test_set, test_tempo) = train_test(drum, bass, tempo, batch_size = batch_size)
drum_set = torch.cat((train_set[0].reshape([-1, 1, 128 * 14 + 16]), test_set[0]), 0)
temp_set = np.concatenate((train_tempo,test_tempo))
result = []
for d in drum_set:
    output = (model(d.view(-1, 128 * 14 + 16).float().to(device)).cpu() > 0.5).float().reshape([128, 36])
    d = d[:, :-16]
#     output = (model(d.view(-1, 128 * 14).float().to(device)).cpu() > 0.5).float().reshape([128, 36])
    d = d.reshape([128, 14]).float()
    result.append(np.array(torch.cat((d, output), 1)))
result = np.array(result)

In [213]:
#output = model(drum_set[15][:, :-16].view(-1, 128 * 14).float().to(device)).cpu()
output = model(drum_set[15].view(-1, 128 * 14 + 16).float().to(device)).cpu() > 0.5

In [216]:
output.min()

tensor(False)

tensor(16., dtype=torch.float64)

In [209]:
import mido
from decode_patterns.data_conversion import build_track, DrumMelodyPair, Converter

converter = Converter((128,50))

# batch_drum = torch.cat((drum_train, drum_test, torch.tensor(drum_validation))).transpose(0,1)
# batch_bass = torch.cat((bass_train.int(), bass_test.int(), torch.tensor(bass_validation).int())).transpose(0,1)
with torch.no_grad():

    bass_outputs = result

    #for i in range(len(bass_outputs)):
    for i in range(0, 100):
        img_dnb = bass_outputs[i]
        t = temp_set[i]
        pair = converter.convert_numpy_image_to_pair(np.array(img_dnb))
#         print(f"pair.melody:{pair.melody}")
        mid = build_track(pair, tempo=t)
        mid.save(f"../midi/model_th_gen/sample{i+1}.mid")
#         np.save(f"midi/npy/drum{i+1}.npy", batch_drum[:,i,:].int())
#         np.save(f"midi/npy/bass{i+1}.npy", bass_outputs[:,i,:])